In [2]:
import glob
import os
import re
import sys

import nltk
import discoursegraphs as dg

In [4]:
raw_filepaths = sorted([f for f in dg.find_files('/home/arne/repos/nishida-nakayama/nlp/resources/corpora/treebank/raw')])
mrg_filepaths = sorted([f for f in dg.find_files('/home/arne/corpora/pennTreebank/parsed/mrg/wsj', '*.mrg')])

In [5]:
def filepath2id(filepath):
    basename = os.path.basename(filepath)
    match = re.search('\d+', basename)
    if match:
        return match.group()
    sys.stderr.write("Can't extract ID from filename: {}".format(basename))

id2mrg = {}
for mf in mrg_filepaths:
    id2mrg[filepath2id(mf)] = mf

Can't extract ID from filename: wsj.all.mrg

In [7]:
def mrg2trees(mrg_filepath):
    mrg_path, mrg_filename = os.path.split(mrg_filepath)
    parsed_doc = nltk.corpus.BracketParseCorpusReader(mrg_path, [mrg_filename])
    return [s for s in parsed_doc.parsed_sents()]

In [6]:
def get_parent_treepos(treepos):
    """Given a treeposition, return the treeposition of its parent."""
    if treepos == ():  # this is the root node
        return None
    return treepos[:-1]

def tree2string_notrace(tree):
    mrg_tokens = tree.leaves()
    raw_tokens = []
    for i, tok in enumerate(mrg_tokens):
        leaf_treepos = tree.leaf_treeposition(i)
        parent_treepos = get_parent_treepos(leaf_treepos)
        if parent_treepos:
            parent_node = tree[parent_treepos]
            if get_nodelabel(parent_node) == '-NONE-':
                continue

        raw_tok = PTB_BRACKET_UNESCAPE.get(tok, tok)
        raw_tokens.append(raw_tok)
    return u' '.join(raw_tokens)

In [8]:
# ([u'!', u'`', u'#', u'%', u'$', u"'", u'&', u'*', u'-', u',', u'.', u'@', u';', u':', u'=', u'?'],)

""" # remove space before/after
! before
` after
#
% before
$ after
' only handle 's
&
*
-
, before
. before
@
; before
: before
=
? before
"""

def untokenize(sent_str):
    # remove space before chars .!?
    re.sub(' (\,|:|\.|!|\?)', r'\1', sent_str)

In [11]:
re.sub(' (\,|;|:|\.|!|\?|%)', r'\1', 'a ! a ? a . a , a ; a : a %')

'a! a? a. a, a; a: a%'

In [17]:
re.sub('(`|\$) ', r'\1', '` hallo, $ 12')

'`hallo, $12'